# This script compares the shape of a group of samples
## Input a dictionary with the location of the samples and their name.
## Also, add the name of the histogram and the binning.

In [22]:
from histogramHelpers import normalization
import ROOT as r
import numpy as np

rootDirectory = "/Users/diegomac/Documents/HEP/VBF-Analysis/Zll/Z-peak/SR/"
channel = "Zll_"
samples = {"MGNLO-RW":rootDirectory+channel+"MGNLORW.root",
           "MG-RW": rootDirectory+channel+"MGRW.root",
           "SherpaNLO-RW": rootDirectory+channel+"SherpaNLORW.root",
           "Sherpa-RW": rootDirectory+channel+"SherpaRW.root"}

colors = {"MGNLO-RW":r.kGreen,
           "MG-RW": r.kGreen+1,
           "SherpaNLO-RW": r.kRed,
           "Sherpa-RW": r.kRed+1}

styles = {"MGNLO-RW":3,
           "MG-RW": 1,
           "SherpaNLO-RW": 3,
           "Sherpa-RW": 1}

histogramName = "mass_jj"
bining = np.array([   0.,  250.,  500.,  750., 1000., 1250., 1500., 2000., 2500., 3000., 4000., 5000.])
normalisationConstant = 250.0

In [23]:
r.TH1.AddDirectory(False)
r.gStyle.SetOptStat(0)

canvas = r.TCanvas("canvas")
canvas.cd()

#canvas.SetLogy()

for sampleName,sampleFile in samples.items():
    file = r.TFile.Open(sampleFile,"READ")
    histogram = file.Get(histogramName)
    histogram.SetDirectory(0)
    histogram = histogram.Rebin(len(bining)-1,sampleName,bining)
    
    integral = histogram.Integral(1,-1)
    histogram.Scale(1.0/integral)
    
    histogram.SetLineColor(colors[sampleName])
    histogram.SetLineStyle(styles[sampleName])
    histogram.SetTitle(sampleName)
    
    normalization([histogram],normalisationConstant)
    
    histogram.Draw("same")
    

canvas.BuildLegend()
canvas.Print("QCDjj.pdf")
    

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file QCDjj.pdf has been created
